#Combine Datasets into one

In [ ]:
import pandas as pd
import csv
import os
import re

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Emotio Dataset

In [ ]:
def get_sentiment_emotions(label):
    if "__label__negative" in label:
        return "negative"
    elif "__label__positive" in label:
        return "positive"
    elif "__label__neutral" in label:
        return "neutral"

In [ ]:
emotions= pd.read_csv("/content/drive/MyDrive/Masterthesis/Data/data_unprocessed/emotions.txt", header=None)
emotions[['label', 'text']] = emotions[0].str.split(' ', n=1, expand=True)
emotions['sentiment'] = emotions['label'].apply(lambda x: get_sentiment_emotions(x))
emotions=emotions[['sentiment', 'text']]
print(f"Length: {len(emotions)}, no. positive: {len(emotions[emotions['sentiment']=='positive'])}, no. negative: {len(emotions[emotions['sentiment']=='negative'])},no. neutral: {len(emotions[emotions['sentiment']=='neutral'])}""")
emotions=emotions[emotions["sentiment"]!="neutral"]

Length: 1306, no. positive: 188, no. negative: 1090,no. neutral: 28


## Prepare df for filmstarts dataset

In [ ]:
def get_sentiment_filmstarts(label):
    if label <3:
        return "negative"
    elif label>3:
        return "positive"
    elif  label==3:
        return "neutral"

In [ ]:
with open("/content/drive/MyDrive/Masterthesis/Data/data_unprocessed/filmstarts/filmstarts.tsv", 'r', newline='', encoding='utf-8') as f:
    dialect = csv.Sniffer().sniff(f.read(2048))
    f.seek(0)
    reader = csv.reader(f, dialect)
    filmstarts = pd.DataFrame(reader)

filmstarts=filmstarts.loc[:,1:]

filmstarts['text']=filmstarts.iloc[:, 1:].apply(lambda row: ' '.join([col for col in row if col is not None]), axis=1)
filmstarts=filmstarts[filmstarts[1].notna()]
filmstarts[1] = filmstarts[1].astype(int)
filmstarts['sentiment'] = filmstarts[1].apply(lambda x: get_sentiment_filmstarts(x))
filmstarts=filmstarts[['sentiment', 'text']]
print(f"Length: {len(filmstarts)}, no. positive: {len(filmstarts[filmstarts['sentiment']=='positive'])}, no. negative: {len(filmstarts[filmstarts['sentiment']=='negative'])},no. neutral: {len(filmstarts[filmstarts['sentiment']=='neutral'])}""")

filmstarts=filmstarts[filmstarts["sentiment"]!="neutral"]

Length: 71126, no. positive: 40015, no. negative: 15611,no. neutral: 15500


<ipython-input-7-4ac888d3d995>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filmstarts[1] = filmstarts[1].astype(int)
<ipython-input-7-4ac888d3d995>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filmstarts['sentiment'] = filmstarts[1].apply(lambda x: get_sentiment_filmstarts(x))


## Prepare df for HOLIDAYS dataset

In [ ]:
def get_sentiment_holidaycheck(label):
    if label <3:
        return "negative"
    elif label>3:
        return "positive"
    elif  label==3:
        return "neutral"


In [ ]:
path="/content/drive/MyDrive/Masterthesis/Data/data_unprocessed/holidaycheck/raw"
file_list = os.listdir(path)


holidaycheck = pd.DataFrame(columns=['label', 'text'])

file_paths = [os.path.join(path, filename) for filename in file_list if filename.endswith('.tsv')]
# Read and concatenate all TSV files
dataframes = []
for file_path in file_paths:
    print(file_path)
    df = pd.read_csv(file_path, sep='\t', header=None, usecols=[1, 2], names=['label', 'text'],on_bad_lines="skip")
    dataframes.append(df)

if dataframes:
    holidaycheck = pd.concat(dataframes, ignore_index=True)
    holidaycheck=holidaycheck.dropna(subset=['label'])
    holidaycheck['sentiment'] = holidaycheck["label"].apply(lambda x: get_sentiment_holidaycheck(x))
    print(f"Length: {len(holidaycheck)}, no. positive: {len(holidaycheck[holidaycheck['sentiment']=='positive'])}, no. negative: {len(holidaycheck[holidaycheck['sentiment']=='negative'])},no. neutral: {len(holidaycheck[holidaycheck['sentiment']=='neutral'])}""")

    holidaycheck=holidaycheck[holidaycheck["sentiment"]!="neutral"]
    holidaycheck=holidaycheck[['sentiment', 'text']]

/content/drive/MyDrive/Masterthesis_clean/Data/data_unprocessed/holidaycheck/raw/holidaycheck.agypt.tsv
/content/drive/MyDrive/Masterthesis_clean/Data/data_unprocessed/holidaycheck/raw/holidaycheck.bulgarien.tsv
/content/drive/MyDrive/Masterthesis_clean/Data/data_unprocessed/holidaycheck/raw/holidaycheck.tunesien.tsv
/content/drive/MyDrive/Masterthesis_clean/Data/data_unprocessed/holidaycheck/raw/holidaycheck.greek.tsv
/content/drive/MyDrive/Masterthesis_clean/Data/data_unprocessed/holidaycheck/raw/holidaycheck.greek2.tsv
/content/drive/MyDrive/Masterthesis_clean/Data/data_unprocessed/holidaycheck/raw/holidaycheck.thai.tsv
/content/drive/MyDrive/Masterthesis_clean/Data/data_unprocessed/holidaycheck/raw/holidaycheck.china.tsv
/content/drive/MyDrive/Masterthesis_clean/Data/data_unprocessed/holidaycheck/raw/holidaycheck.mexico.tsv
/content/drive/MyDrive/Masterthesis_clean/Data/data_unprocessed/holidaycheck/raw/holidaycheck.indien.tsv
/content/drive/MyDrive/Masterthesis_clean/Data/data_unp

## Combine into one

In [ ]:
data = pd.concat([emotions, filmstarts, holidaycheck], ignore_index=True)
#combine datasets into one

output_path = "/content/drive/MyDrive/Masterthesis/Data/data_unprocessed/combined.tsv"
data.to_csv(output_path, sep='\t', index=False)